# 9) Asignar etiquetas numéricas al shapefile dado que trae etiquetas

In [66]:
path_shapefile_with_classes = '/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/puntos_con_etiquetas/'

In [67]:
input_file = path_shapefile_with_classes + 'shp_puntos_martinez_lcc.shp'

In [68]:
input_file

'/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/puntos_con_etiquetas/shp_puntos_martinez_lcc.shp'

In [69]:
with fiona.open(input_file) as src:
    fc=list(src)
    crs = to_string(src.crs)

In [70]:
fc[0]

{'type': 'Feature',
 'id': '0',
 'properties': OrderedDict([('CVE_MUN', '124'),
              ('Municip', 'Papantla'),
              ('DstrtDR', 'Martínez de la Torre'),
              ('Ecoregn', 'Veracruz moist forests'),
              ('ZonadVd', 'Bosque Subhúmedo premontano [Cálido - Lluvioso]'),
              ('Name', 'Maíz'),
              ('Obsrvcn', None)]),
 'geometry': {'type': 'Point',
  'coordinates': (2987250.8825278645, 932708.3126494994)}}

In [71]:
name_attribute='Name'

In [72]:
fc_subset = [{'id': feature['id'],
              'name': feature['properties'][name_attribute],
              'geometry': feature['geometry']} for feature in fc]

In [73]:
fc_subset[0]

{'id': '0',
 'name': 'Maíz',
 'geometry': {'type': 'Point',
  'coordinates': (2987250.8825278645, 932708.3126494994)}}

In [74]:
def normalize_name_classes(string):
    return unicodedata.normalize('NFKD', string).encode('ASCII','ignore').decode('utf-8').lower()

In [77]:
for k in range(len(fc_subset)):
    fc_subset[k]['name'] =  normalize_name_classes(fc_subset[k]['name'])

In [78]:
fc_subset[0]

{'id': '0',
 'name': 'maiz',
 'geometry': {'type': 'Point',
  'coordinates': (2987250.8825278645, 932708.3126494994)}}

In [79]:
classes = list(set([feature['name'] for feature in fc_subset]))

In [80]:
classes

['cafe', 'pastizal', 'citrico', 'mango', 'platano', 'maiz', 'cafe sol']

In [81]:
len(classes)

7

In [82]:
def add_numeric_code_to_classes(classes):
    return {classes[k]: k for k in range(len(classes))}

In [83]:
classes = add_numeric_code_to_classes(classes)

In [545]:
classes

{'pastizal': 0,
 'maiz': 1,
 'mango': 2,
 'platano': 3,
 'cafe': 4,
 'citrico': 5,
 'cafe sol': 6}

# 10) Insertar en madmex_Tag y añadir numeric_code a fc_subset

**Verificar credenciales de la DB a la que se insertará**

In [88]:
scheme_tag = 'agricultura_completo'

In [89]:
[Tag.objects.get_or_create(scheme=scheme_tag, value=name_class, numeric_code=classes[name_class],color='') for name_class in classes]

[(<Tag: Tag object (54)>, False),
 (<Tag: Tag object (55)>, False),
 (<Tag: Tag object (56)>, False),
 (<Tag: Tag object (57)>, False),
 (<Tag: Tag object (58)>, False),
 (<Tag: Tag object (59)>, False),
 (<Tag: Tag object (60)>, False)]

In [90]:
def get_numeric_code(string):
    return classes[string]

In [91]:
fc_subset = [{'id': feature['id'],
               'name': feature['name'],
               'code': get_numeric_code(feature['name']),
               'geometry': feature['geometry']} for feature in fc_subset]

In [92]:
fc_subset[10]

{'id': '10',
 'name': 'cafe sol',
 'code': 6,
 'geometry': {'type': 'Point',
  'coordinates': (3013642.151646747, 880227.5095511458)}}

In [93]:
fc_subset_schema = {'geometry': 'Point',
                    'properties': {'name': 'str',
                                   'code': 'int'}
                   }

In [94]:
output_file_with_numeric_codes = path_shapefile_with_classes + 'shp_puntos_martinez_with_numeric_code_lcc.shp'

In [95]:
output_file_with_numeric_codes

'/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/puntos_con_etiquetas/shp_puntos_martinez_with_numeric_code_lcc.shp'

In [96]:
driver='ESRI Shapefile'

In [97]:
layer='shp_puntos_martinez_with_numeric_code_lcc'

In [556]:
with fiona.open(output_file_with_numeric_codes, 'w',
                encoding='utf-8',
                driver=driver,
                layer=layer,
                crs=crs,
                schema=fc_subset_schema) as dst:
    [dst.write({'geometry': feature['geometry'],
                'properties': {'name': feature['name'],
                               'code': feature['code']}
                }) for feature in fc_subset]

# 11) Registrar cultivos con etiquetas numéricas en trainclassification con esquema "agriculture_completo"

In [5]:
%%bash
/home/madmex_user/.local/bin/antares ingest_training_from_vector /LUSTRE/MADMEX/training/cultivos/igamma_veracruz/puntos_con_etiquetas/shp_puntos_martinez_with_numeric_code_lcc.shp --scheme agricultura_completo --year 2020 --name train_cultivos_inecol --field code

Command line arguments: {'verbosity': 1, 'settings': None, 'pythonpath': None, 'traceback': False, 'no_color': False, 'force_color': False, 'input_file': '/LUSTRE/MADMEX/training/cultivos/igamma_veracruz/puntos_con_etiquetas/shp_puntos_martinez_with_numeric_code_lcc.shp', 'scheme': 'agricultura_completo', 'field': 'code', 'name': 'train_cultivos_inecol', 'year': '2020', 'app': False}
Command execution is done in 15.899461030960083 seconds.
2020-02-05 19:40:43,192 - madmex.management.base - base execute: Command execution is done in 15.899461030960083 seconds.
